In [1]:
import sys,os,argparse,time
import numpy as np
import torch
import utils
from datetime import datetime
import pickle
tstart=time.time()

In [2]:
# Arguments
parser=argparse.ArgumentParser(description='xxx')
parser.add_argument('--seed',               default=0,              type=int,     help='(default=%(default)d)')
parser.add_argument('--device',             default='cpu',          type=str,     help='gpu id')
parser.add_argument('--experiment',         default='16_task_groups',       type =str,    help='Mnist or dissertation')
parser.add_argument('--approach',           default='PUGCL',          type =str,    help='Method, always Lifelong Uncertainty-aware learning')
parser.add_argument('--data_path',          default='data/data.csv',     type=str,     help='gpu id')

# Training parameters
parser.add_argument('--output',             default='',             type=str,     help='')
parser.add_argument('--checkpoint_dir',     default='../checkpoints_16_tasks',    type=str,   help='')
parser.add_argument('--n_epochs',           default=100,              type=int,     help='')
parser.add_argument('--batch_size',         default=64,             type=int,     help='')
parser.add_argument('--lr',                 default=0.001,           type=float,   help='')
parser.add_argument('--hidden_size',        default=800,           type=int,     help='')
parser.add_argument('--parameter',          default='',             type=str,     help='')

# UCB HYPER-PARAMETERS
parser.add_argument('--MC_samples',         default='10',           type=int,     help='Number of Monte Carlo samples')
parser.add_argument('--rho',                default='-3',           type=float,   help='Initial rho')
parser.add_argument('--sigma1',             default='0.0',          type=float,   help='STD foor the 1st prior pdf in scaled mixture Gaussian')
parser.add_argument('--sigma2',             default='6.0',          type=float,   help='STD foor the 2nd prior pdf in scaled mixture Gaussian')
parser.add_argument('--pi',                 default='0.25',         type=float,   help='weighting factor for prior')

parser.add_argument('--resume',             default='no',           type=str,     help='resume?')
parser.add_argument('--sti',                default=1,              type=int,     help='starting task?')

parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args=parser.parse_args()
utils.print_arguments(args)

Arguments =
	seed: 0
	device: cpu
	experiment: 16_task_groups
	approach: PUGCL
	data_path: data/data.csv
	output: 
	checkpoint_dir: ../checkpoints_16_tasks
	n_epochs: 100
	batch_size: 64
	lr: 0.001
	hidden_size: 800
	parameter: 
	MC_samples: 10
	rho: -3.0
	sigma1: 0.0
	sigma2: 6.0
	pi: 0.25
	resume: no
	sti: 1
	fff: /Users/jonastjomsland/Library/Jupyter/runtime/kernel-003dd4ae-5b1e-4b75-9dcd-182fa5aa8112.json


In [3]:
# Set seed for stable results
np.random.seed(args.seed)
torch.manual_seed(args.seed)

# Check if Cuda is available
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("Using device:", args.device)

# Checkpoint
checkpoint = utils.make_directories(args)
args.checkpoint = checkpoint
print()

# PUGCL with two tasks:
from data import dataloader_16_tasks as dataloader

# Import Lifelong Uncertainty-aware Learning approach:
#from bayesian_model.lul import Lul
from training_method import PUGCL

# Import model used:
#from bayesian_model.bayesian_network import BayesianNetwork
from bayesian_model.bayesian_network import BayesianNetwork

Using device: cpu
16_task_groups_PUGCL
Results will be saved in  ../checkpoints_16_tasks/16_task_groups_PUGCL



In [4]:
print()
print("Starting this session on: ")
print(datetime.now().strftime("%Y-%m-%d %H:%M"))

# Load data:
print("Loading data...")
data, task_outputs, input_size = dataloader.get(data_path=args.data_path)
print("Input size =", input_size, "\nTask info =", task_outputs)
print("Number of data samples: ", len(data[0]['train']['x']))
args.num_tasks = len(task_outputs)
args.input_size = input_size
args.task_outputs = task_outputs
pickle.dump(data, open( "data/data.p", "wb" ))

# Initialize Bayesian network
print("Initializing network...")
model = BayesianNetwork(args).to(args.device)

# Initialize Lul approach
print("Initialize Lifelong Uncertainty-aware Learning")
approach = PUGCL(model, args=args)
print("-"*100)

# Check wether resuming:
if args.resume == "yes":
    checkpoint = torch.load(os.path.join(args.checkpoint, 'model_{}.pth.tar'.format(args.sti)))
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device=args.device)
else:
    args.sti = 0


Starting this session on: 
2020-05-13 15:21
Loading data...
Input size = [1, 29] 
Task info = [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2)]
Number of data samples:  500
Initializing network...
Initialize Lifelong Uncertainty-aware Learning
----------------------------------------------------------------------------------------------------


/Users/jonastjomsland/anaconda3/envs/ucb/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [ ]:
# Iterate over the two tasks:
loss = np.zeros((len(task_outputs), len(task_outputs)), dtype=np.float32)
for task, n_class in task_outputs[args.sti:]:
    print('*'*100)
    print('Task {:2d} ({:s})'.format(task, data[task]['name']))
    print('*'*100)

    # Get data:
    xtrain = data[task]['train']['x'][:,1:].type(torch.float32).to(args.device)
    ytrain = data[task]['train']['y'].type(torch.float32).to(args.device)

    # Start training
    print("Starting training for the tasks in group: ", task)
    approach.train(task, xtrain, ytrain)
    print('_'*100)

    # Test for this task group:
    for u in range(task+1):
        xtest = data[u]['test']['x'][:,1:].type(torch.float32).to(args.device)
        ytest = data[u]['test']['y'].type(torch.float32).to(args.device)
        test_loss = approach.eval(u, xtest, ytest, debug=True)
        print("Test on task {:2d} - {:15s}: loss={:.3f}".format(u, data[u]['name'], test_loss))
        loss[task, u] = test_loss

    # Save
    print("Saving at " + args.checkpoint)
    np.savetxt(os.path.join(args.checkpoint, '{}_{}_{}.txt'.format(args.experiment, "Lul", args.seed)), loss, '%.5f')

****************************************************************************************************
Task  0 (Vacuum cleaning)
****************************************************************************************************
Starting training for the tasks in group:  0
Batch: 0/450 

../torch/csrc/utils/python_arg_parser.cpp:750: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)


Batch: 448/450 | Epoch   1, time=696.9ms/ 13.1ms | Train: loss=1.858 | Valid: loss=1.935 | *
Batch: 448/450 | Epoch   2, time=659.0ms/ 13.2ms | Train: loss=1.662 | Valid: loss=1.641 | *
Batch: 448/450 | Epoch   3, time=655.1ms/ 12.7ms | Train: loss=1.666 | Valid: loss=1.610 | *
Batch: 448/450 | Epoch   4, time=658.3ms/ 13.1ms | Train: loss=1.649 | Valid: loss=1.612 |
Batch: 448/450 | Epoch   5, time=656.2ms/ 14.3ms | Train: loss=1.704 | Valid: loss=1.640 |
Batch: 448/450 | Epoch   6, time=663.8ms/ 12.8ms | Train: loss=1.624 | Valid: loss=1.577 | *
Batch: 448/450 | Epoch   7, time=655.3ms/ 14.9ms | Train: loss=1.649 | Valid: loss=1.626 |
Batch: 448/450 | Epoch   8, time=656.5ms/ 12.6ms | Train: loss=1.618 | Valid: loss=1.551 | *
Batch: 448/450 | Epoch   9, time=667.3ms/ 15.4ms | Train: loss=1.629 | Valid: loss=1.538 | *
Batch: 448/450 | Epoch  10, time=710.4ms/ 15.5ms | Train: loss=1.558 | Valid: loss=1.480 | *
Batch: 448/450 | Epoch  11, time=669.0ms/ 13.6ms | Train: loss=1.564 | Valid

Batch: 448/450 | Epoch  91, time=670.6ms/ 13.2ms | Train: loss=1.340 | Valid: loss=1.291 |
Batch: 448/450 | Epoch  92, time=689.8ms/ 15.3ms | Train: loss=1.390 | Valid: loss=1.342 |
Batch: 448/450 | Epoch  93, time=660.0ms/ 15.8ms | Train: loss=1.330 | Valid: loss=1.285 |
Batch: 448/450 | Epoch  94, time=680.8ms/ 14.5ms | Train: loss=1.341 | Valid: loss=1.288 |
Batch: 448/450 | Epoch  95, time=678.1ms/ 13.9ms | Train: loss=1.443 | Valid: loss=1.365 |
Batch: 448/450 | Epoch  96, time=684.6ms/ 15.7ms | Train: loss=1.411 | Valid: loss=1.342 |
Batch: 448/450 | Epoch  97, time=681.3ms/ 14.8ms | Train: loss=1.482 | Valid: loss=1.506 |
Batch: 448/450 | Epoch  98, time=662.1ms/ 12.7ms | Train: loss=1.323 | Valid: loss=1.278 | *
Batch: 448/450 | Epoch  99, time=650.8ms/ 12.7ms | Train: loss=1.334 | Valid: loss=1.309 |
Batch: 448/450 | Epoch 100, time=678.4ms/ 13.6ms | Train: loss=1.336 | Valid: loss=1.298 |
________________________________________________________________________________________